In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D,Dropout
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#inspiration from https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
!pip install pyspellchecker

In [ ]:
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
# def clean_text(
#     string: str, 
#     punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
#     stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
#     """
#     A method to clean text 
#     """
#     # Cleaning the urls
#     string = re.sub(r'https?://\S+|www\.\S+', '', string)

#     # Cleaning the html elements
#     string = re.sub(r'<.*?>', '', string)

#     # Removing the punctuations
#     for x in string.lower(): 
#         if x in punctuations: 
#             string = string.replace(x, "") 

#     # Converting the text to lower
#     string = string.lower()

#     # Removing stop words
#     string = ' '.join([word for word in string.split() if word not in stop_words])

#     # Cleaning the whitespaces
#     string = re.sub(r'\s+', ' ', string).strip()

#     return correct_spellings(remove_emoji(string))




In [ ]:
df=pd.concat([train_df,test_df])

In [ ]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

In [ ]:
df['text']=df['text'].apply(lambda x : remove_html(x))

In [ ]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [ ]:
#df['text']=df['text'].apply(lambda x : correct_spellings(x))
#df.to_csv("/kaggle/working/df.csv")
#execute only first time
df=pd.read_csv("../input/processed-nlp-data/processed_data.csv")

In [ ]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus(df)

In [ ]:
Y=np.zeros(50)
X=[len(i) for i in corpus]
for i in X:
    if i<50:
        Y[i]+=1
plt.bar(range(50),Y)
plt.show()

In [ ]:
embedding_dict={}
with open('../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
MAX_LEN=30
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,200))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
            

In [ ]:
model=Sequential()

embedding=Embedding(num_words,200,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(124, dropout=0.1, recurrent_dropout=0.1))
model.add(LSTM(64))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))


optimzer=tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [ ]:

model.summary()


In [ ]:
train=tweet_pad[:train_df.shape[0]]
test=tweet_pad[train_df.shape[0]:]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,train_df['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=4,  restore_best_weights=True)

history=model.fit(X_train,y_train,epochs=15,callbacks=[es],validation_data=(X_test,y_test))

In [ ]:
sample_sub=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
y_pre=model.predict(test)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission.csv',index=False)

In [ ]:
sub.head()

In [ ]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "memes"